In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1]:
%run ../Jane/extract_data.py

In [5]:
airqual_data.groupby(['State','County']).count()
# if n_date != n_AQIMean, there are missing values present in the data

Date  AQIMean
State County                             
CA    Alameda County         127      127
      Contra Costa County    127      127
      Marin County           127      127
      Napa County            127      127
      San Francisco County   127      127
      San Mateo County       127      127
      Santa Clara County     126      126
      Solano County          126      126
      Sonoma County          127      127
FL    Broward County         126      126
      Miami-Dade County      126      126
      Palm Beach County       97       97
NY    Bronx County           129      129
      Kings                  129      129
      New York               129      129
      Queens County          127      127
      Richmond               129      129
TX    Travis County          126      126

In [98]:
# make sure airqual_data has been transformed by extract_data.py

nyc_avg = airqual_data[airqual_data.City=='New York'].groupby('Date').mean().reset_index()
nyc_avg['State'] = 'New York'
nyc_avg['City'] = 'New York'
nyc_aq = pd.concat((nyc_avg,nyc_avg,nyc_avg))
nyc_aq['County']= counties
nyc_aq = nyc_aq[['State','County','City','AQIMean','Date']]

airqual_data = pd.concat((airqual_data,nyc_aq))

,Date,AQIMean,State,City
0,2010-01-01,24.638826,New York,New York
1,2010-02-01,23.706344,New York,New York
2,2010-03-01,23.670713,New York,New York
3,2010-04-01,22.261414,New York,New York
4,2010-05-01,20.099075,New York,New York


In [ ]:
# SF Metro
counties = ['Alameda County', 'Contra Costa County', 'Marin County', 'Napa County', 'San Mateo County', 
            'Santa Clara County', 'Solano County', 'Sonoma County', 'San Francisco County']
# NY Metro:
counties = ['New York County', 'Bronx County', 'Queens County', 'Kings County', 'Richmond County']
# Greater Austin Metro:
counties = ['Travis County']
# Miami Metro:
counties = ['Miami-Dade County', 'Broward County', 'Palm Beach County']

In [130]:
# working with a smaller subset to explore
nyc_zillow = zillow_data[zillow_data.City == 'New York'].copy()
nyc_zipcodes = zillow_data[zillow_data.City == 'New York'].Zipcode.value_counts().reset_index()['index'].to_list()
nyc_airqual = airqual_data[airqual_data['City'] == 'New York'].copy()
nyc_persinc = persinc_data[persinc_data.City == 'New York'].copy()
nyc_census = census_data[census_data.Zipcode.isin(nyc_zipcodes)].copy()

In [132]:
nyc_zillow.drop(['RegionID','Metro'],axis=1,inplace=True)
# we want to drop the zipcode 10069
nyc_zillow = nyc_zillow[nyc_zillow.Zipcode != '10069'].copy()

In [136]:
nyc_zillow[nyc_zillow['Year'] >= 2015]

,Zipcode,City,State,County,SizeRank,Date,Rent,Year
96772,10025,New York,NY,New York County,1,2015-01-01,3566.0,2015
96774,10023,New York,NY,New York County,3,2015-01-01,3413.0,2015
96778,10002,New York,NY,New York County,7,2015-01-01,3508.0,2015
96782,11226,New York,NY,Kings County,11,2015-01-01,1876.0,2015
96783,10467,New York,NY,Bronx County,12,2015-01-01,1442.0,2015
...,...,...,...,...,...,...,...,...
210249,11416,New York,NY,Queens County,1818,2020-01-01,2184.0,2020
210251,11436,New York,NY,Queens County,1820,2020-01-01,2057.0,2020
210257,11366,New York,NY,Queens County,1826,2020-01-01,2185.0,2020
210259,11109,New York,NY,New York County,1828,2020-01-01,3325.0,2020


In [5]:
# reshape for better merging
inclvl_moderate = inclvl_data[inclvl_data.income_level_group == 'Moderate'][['Date','Vol']]
inclvl_moderate = inclvl_moderate.rename(columns={'Vol':'Vol_moderate_income'})
inclvl_low = inclvl_data[inclvl_data.income_level_group == 'Low'][['Date','Vol']]
inclvl_low = inclvl_low.rename(columns={'Vol':'Vol_low_income'})
inclvl_data = pd.merge(inclvl_moderate,inclvl_low,on='Date')
inclvl_data.head()

,Date,Vol_moderate_income,Vol_low_income
0,2005-01-01,4.578106e+09,7.635135e+08
1,2005-02-01,4.563908e+09,7.558710e+08
2,2005-03-01,4.762458e+09,8.171551e+08
3,2005-04-01,4.744005e+09,7.854169e+08
4,2005-05-01,4.752915e+09,8.289172e+08


In [44]:
df = pd.merge(nyc_zillow, inclvl_data,how='left',on='Date')

In [53]:
nyc_airqual.drop(['State','City'],axis=1,inplace=True)

In [48]:
df = df.merge(nyc_persinc, 'left', on ='Year')

In [142]:
acs = census_data[['do_date','total_pop','households','median_age','median_income','income_per_capita','pop_determined_poverty_status',
                  'poverty','gini_index','housing_units','different_house_year_ago_different_city','different_house_year_ago_same_city',
                  'pop_in_labor_force','aggregate_travel_time_to_work','bachelors_degree','employed_pop','unemployed_pop',
                  'employed_arts_entertainment_recreation_accommodation_food','Zipcode']].copy()

In [144]:
# pct poverty
acs['pct_poverty'] = acs['poverty']/acs['pop_determined_poverty_status']
acs.drop(['poverty','pop_determined_poverty_status'],axis=1,inplace=True)

# estimated number of homes for each household (scarse? or many available?)
acs['housing_availability'] = acs['housing_units']/acs['households']
# estimated number of people per household
acs['home_density'] = acs['total_pop']/acs['households']

acs['pct_employed'] = acs['employed_pop']/acs['pop_in_labor_force']
acs['pct_jobs_nightlife'] = acs['employed_arts_entertainment_recreation_accommodation_food']/acs['employed_pop']
acs['pct_unemployed'] = acs['unemployed_pop']/acs['pop_in_labor_force']

# have moved from somewhere else in the same city
acs['move_within_city'] = acs['different_house_year_ago_same_city']/acs['total_pop']
# have moved to this city from a new city
acs['move_new_city'] = acs['different_house_year_ago_different_city']/acs['total_pop']
acs['avg_commute_time'] = acs['aggregate_travel_time_to_work'] / acs['employed_pop']

acs.drop(['pop_in_labor_force','housing_units','employed_pop','employed_arts_entertainment_recreation_accommodation_food','unemployed_pop',
          'different_house_year_ago_same_city','different_house_year_ago_different_city','aggregate_travel_time_to_work'],axis=1,inplace=True)

# percent of population w/ bachelors degree
acs['pct_college'] = acs['bachelors_degree'] / acs['total_pop']
acs.drop('bachelors_degree',axis=1,inplace=True)

In [149]:
acs.sample(10)

,total_pop,households,median_age,median_income,income_per_capita,gini_index,Zipcode,pct_poverty,housing_availability,home_density,pct_employed,pct_jobs_nightlife,pct_unemployed,move_within_city,move_new_city,avg_commute_time,pct_college
9490,2654,1049,45.1,33363.0,18027.0,0.4134,36850,0.254539,1.334604,2.530029,0.915544,0.030422,0.084456,0.013564,0.036549,30.853778,0.048983
4772,1859,629,55.5,41010.0,18881.0,0.3158,18321,0.091985,1.311606,2.955485,0.948916,0.110930,0.051084,0.000000,0.155460,NaN,0.059172
15161,40402,15055,33.0,53421.0,26766.0,0.4313,21804,0.176364,1.119296,2.683627,0.932784,0.140935,0.063841,0.073957,0.143929,20.405392,0.096975
17653,699,306,47.6,70147.0,29383.0,0.3080,43323,0.022890,1.032680,2.284314,0.959770,0.056886,0.040230,0.001431,0.027182,19.970060,0.055794
31663,321,163,41.0,44019.0,20830.0,0.2090,23488,0.239875,1.000000,1.969325,0.954082,0.000000,0.045918,0.000000,0.000000,NaN,0.028037
24979,45895,17227,37.2,70842.0,35323.0,0.4305,55112,0.095792,1.027631,2.664132,0.952753,0.084588,0.047247,0.016037,0.133479,19.965713,0.194095
4697,1131,360,35.0,48214.0,21138.0,0.3930,47147,0.160035,1.069444,3.141667,0.975610,0.000000,0.024390,0.000000,0.015915,NaN,0.026525
25107,17731,6663,34.8,63414.0,30857.0,0.4223,33559,0.120415,1.100855,2.661114,0.939027,0.118875,0.057849,0.029891,0.128983,28.821839,0.143252
4084,1829,1011,55.8,51738.0,46258.0,0.4747,55384,0.061335,1.108803,1.809100,0.965352,0.059406,0.034648,0.008748,0.215965,25.068069,0.202296
32259,159,101,65.1,51913.0,43443.0,0.1561,96116,0.000000,2.009901,1.574257,1.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000


In [146]:
acs.drop('do_date',axis=1,inplace=True)

In [82]:
census_feats = acs.drop('Zipcode',axis=1)

In [100]:
census_feats.shape

(149, 16)